In [8]:
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests

from acquire import scrape_github_data
from prepare import words

from env import github_token, github_username

In [3]:
# Imports for modeling... and stuff

import pandas as pd
from prepare import basic_clean, tokenize, stem, lemmatize, remove_stopwords, prep_article_data, words
from acquire import scrape_github_data

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 250)

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
import re

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split

# Import Decision Tree and Random Forest ;D
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [4]:
#Run scraping function
# data = scrape_github_data()

In [7]:
#Turn scraped data into raw df
# df = pd.DataFrame(data)
df = pd.read_json('data.json')



#Check returned df 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   repo             100 non-null    object
 1   language         96 non-null     object
 2   readme_contents  100 non-null    object
dtypes: object(3)
memory usage: 2.5+ KB


In [9]:
df = words(df)

In [10]:
df.language.value_counts()

other         53
Python        18
C++           17
JavaScript    12
Name: language, dtype: int64

### Modeling prep

In [11]:
# Make copies of df with prepared columns and target
clean_df = df.copy()[['language', 'clean']]
stem_df = df.copy()[['language', 'stemmed']]
lem_df = df.copy()[['language', 'lemmatized']]


# Get splits for each of the above dfs and isolate target
X_clean = clean_df[['clean']]
y_clean = clean_df.language

X_clean_train, X_clean_test, y_clean_train, y_clean_test = train_test_split(X_clean, y_clean, test_size=.2, random_state=302)
X_clean_train, X_clean_validate, y_clean_train, y_clean_validate  = train_test_split(X_clean_train, y_clean_train, test_size=.3, random_state=302)

print(X_clean_train.shape, X_clean_validate.shape, X_clean_test.shape)

X_stem = stem_df[['stemmed']]
y_stem = stem_df.language

X_stem_train, X_stem_test, y_stem_train, y_stem_test = train_test_split(X_stem, y_stem, test_size=.2, random_state=302)
X_stem_train, X_stem_validate, y_stem_train, y_stem_validate  = train_test_split(X_stem_train, y_stem_train, test_size=.3, random_state=302)

print(X_stem_train.shape, X_stem_validate.shape, X_stem_test.shape)

X_lem = lem_df[['lemmatized']]
y_lem = lem_df.language

X_lem_train, X_lem_test, y_lem_train, y_lem_test = train_test_split(X_lem, y_lem, test_size=.2, random_state=302)
X_lem_train, X_lem_validate, y_lem_train, y_lem_validate  = train_test_split(X_lem_train, y_lem_train, test_size=.3, random_state=302)

print(X_lem_train.shape, X_lem_validate.shape, X_lem_test.shape)




(56, 1) (24, 1) (20, 1)
(56, 1) (24, 1) (20, 1)
(56, 1) (24, 1) (20, 1)


### Modeling

In [12]:
# "Clean" models
cv = CountVectorizer()
tfidf = TfidfVectorizer()

cv_clean_bow = cv.fit_transform(X_clean_train.clean)

tf_clean_bow = tfidf.fit_transform(X_clean_train.clean)


# Check cv bag of words
cv_clean_bow.todense()


matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 1, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [13]:
# Check tfidf bag of words
tf_clean_bow.todense()

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.01511162, ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

In [14]:
# Make and fit decision tree object for cv_clean_bow
cv_tree1 = DecisionTreeClassifier(max_depth=5)
cv_tree1.fit(cv_clean_bow, y_clean_train)
#Get tree score
cv_tree1.score(cv_clean_bow, y_clean_train)
#Make and fit decision tree object for tf_clean_bow
tf_tree1 = DecisionTreeClassifier(max_depth=5)
tf_tree1.fit(tf_clean_bow, y_clean_train)

#Get tree score
tf_tree1.score(tf_clean_bow, y_clean_train)

0.9107142857142857

In [15]:
# "Stemmed" models

# Bags
cv_stem_bow = cv.fit_transform(X_stem_train.stemmed)
tf_stem_bow = tfidf.fit_transform(X_stem_train.stemmed)

# Make and fit decision tree object for cv_stem_bow
cv_tree2 = DecisionTreeClassifier(max_depth=5)
cv_tree2.fit(cv_stem_bow, y_stem_train)

#Make and fit decision tree object for tf_stem_bow
tf_tree2 = DecisionTreeClassifier(max_depth=5)
tf_tree2.fit(tf_stem_bow, y_stem_train)



DecisionTreeClassifier(max_depth=5)

In [16]:
#Get tree score
cv_tree2.score(cv_stem_bow, y_stem_train)

0.8571428571428571

In [17]:
#Get tree score
tf_tree2.score(tf_stem_bow, y_stem_train)

0.9107142857142857

In [18]:
# "Lemmatized" models

cv_lem_bow = cv.fit_transform(X_lem_train.lemmatized)
tf_lem_bow = tfidf.fit_transform(X_lem_train.lemmatized)

# Make and fit decision tree object for cv_lem_bow
cv_tree3 = DecisionTreeClassifier(max_depth=5)
cv_tree3.fit(cv_lem_bow, y_lem_train)

#Make and fit decision tree object for tf_lem_bow
tf_tree3 = DecisionTreeClassifier(max_depth=5)
tf_tree3.fit(tf_lem_bow, y_lem_train)

DecisionTreeClassifier(max_depth=5)

In [19]:
# Output tree scores
print(f'CV tree score: {cv_tree3.score(cv_lem_bow, y_lem_train)}\nTFIDF tree score: {tf_tree3.score(tf_lem_bow, y_lem_train)}')

CV tree score: 0.8571428571428571
TFIDF tree score: 0.9107142857142857


In [20]:
dec_tree_training_scores= {
    'CV_clean': cv_tree1.score(cv_clean_bow, y_clean_train),
    'CV_stem': cv_tree2.score(cv_stem_bow, y_stem_train),
    'CV_lem': cv_tree3.score(cv_lem_bow, y_lem_train),
    'TFIDF_clean': tf_tree1.score(tf_clean_bow, y_clean_train),
    'TFIDF_stem': tf_tree2.score(tf_stem_bow, y_stem_train),
    'TFIDF_lem': tf_tree3.score(tf_lem_bow, y_lem_train)
}

dec_tree_training_scores
#pd.DataFrame(dec_tree_training_scores, index=)

{'CV_clean': 0.8571428571428571,
 'CV_stem': 0.8571428571428571,
 'CV_lem': 0.8571428571428571,
 'TFIDF_clean': 0.9107142857142857,
 'TFIDF_stem': 0.9107142857142857,
 'TFIDF_lem': 0.9107142857142857}

### Takeaways: Both vectorizers scoring the same on clean, stemmed, and lemmatized text

In [21]:
#Transform validate split with vectorizer
tf_clean_bow_val = tfidf.transform(X_clean_validate.clean)

#Get tf_tree1 score on validate
tf_tree1.score(tf_clean_bow_val, y_clean_validate)

ValueError: X has 6162 features, but DecisionTreeClassifier is expecting 6495 features as input.

In [19]:
#Transform validate split with vectorizer
tf_stem_bow_val = tfidf.transform(X_stem_validate.stemmed)

# Validate TFIDF tree
#tf_tree5 = DecisionTreeClassifier(max_depth=5)
#tf_tree5.fit(tf_stem_bow_val, y_stem_validate)

#Get tf_tree2 score on validate
tf_tree2.score(tf_stem_bow_val, y_stem_validate)

ValueError: X has 6162 features, but DecisionTreeClassifier is expecting 5585 features as input.

In [40]:
#Transform validate split with vectorizer
tf_lem_bow_val = tfidf.transform(X_lem_validate.lemmatized)



#Get tf_tree3 score on validate
tf_tree3.score(tf_lem_bow_val, y_lem_validate)

1.0

### Scores went up?? Something might be wrong...